In [1]:
import pandas as pd

import networkx as nx
from networkx.algorithms import bipartite

In [2]:
with open('day-16-input.txt', 'r') as f:
    rules, my, others = f.read().split('\n\n')
    rules = dict([l.split(': ') for l in rules.split('\n')])
    rules = {k:[[int(a) for a in r.split('-')] for r in v.split(' or ')] for k, v in rules.items()}
    rules = {k:tuple([tuple(a) for a in v]) for k, v in rules.items()}
    my = [int(a) for a in my.split('\n')[1].split(',')]
    others = [[int(a) for a in l.split(',')] for l in others.split('\n')[1:]]

# Part 1

In [3]:
flat = [f for l in others for f in l]

n = 0
for f in flat:
    passes = False
    for (l1, r1), (l2, r2) in rules.values():
        if l1 <= f <= r1 or l2 <= f <= r2:
            passes = True
            break
    n += f * int(not passes)

print(f'Part 1: {n}')

Part 1: 23115


# Part 2

In [4]:
valid = []
for l in [my] + others:
    passes = True
    for e in l:
        a = False
        for (l1, r1), (l2, r2) in rules.values():
            if l1 <= e <= r1 or l2 <= e <= r2:
                a = True
                break
        if not a:
            passes = False
    if passes:
        valid.append(l)

valid = pd.DataFrame(valid)
unique = [set(valid[col].unique()) for col in valid.columns]

In [5]:
possible = {}
for r, ((l1, r1), (l2, r2)) in rules.items():
    p = []
    for i, l in enumerate(unique):
        works = True
        for e in l:
            if not (l1 <= e <= r1 or l2 <= e <= r2):
                works = False
        if works:
            p.append(i)
    possible[r] = p

In [6]:
G = nx.Graph()
G.add_nodes_from(possible.keys(), bipartite=0)
G.add_nodes_from(range(len(my)), bipartite=1)

for name, l in possible.items():
    for col in l:
        G.add_edge(name, col)

matches = bipartite.matching.hopcroft_karp_matching(G, possible.keys())

In [7]:
order = [None] * len(my)
for n, i in matches.items():
    try:
        order[i] = n
    except:
        pass

In [8]:
res = 1
for i, n in enumerate(order):
    if n.startswith('departure'):
        res *= my[i]
print(f'Part 2: {res}')

Part 2: 239727793813
